In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd  /content/drive/MyDrive/online arduino kit - cv group
!pwd

/content/drive/.shortcut-targets-by-id/12qdxIhrJkYThSDH6ulLJ4IaUDG4pGrFG/RA_Daragh_CV_Arduino
/content/drive/.shortcut-targets-by-id/12qdxIhrJkYThSDH6ulLJ4IaUDG4pGrFG/RA_Daragh_CV_Arduino


In [ ]:
import os
import cv2

# Extrat frames from the videos

To prepare the video folder, you should create a folder for each class/component and put all the videos of it in that folder.

Change the parameters in  cells below accordingly.


1.   **NUM_FRAMES_PER_SECOND**  
2.   **input_folder**  
3.   **output_folder**  




Note if **NUM_FRAMES_PER_SECOND** is too high, it will take longer for the model to train the same epoch (but the model may converge at ealier epoch too).

In [ ]:
#### CHANGE ACCORDINGLY###
NUM_FRAMES_PER_SECOND = 3 # Define how many frames you want to extract from the video
#### CHANGE ACCORDINGLY###


def extract_frames(count, video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = int(fps / NUM_FRAMES_PER_SECOND)

    # Extract frames
    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if idx % interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{count}.jpg")
            cv2.imwrite(frame_path, frame)
            count += 1
        idx += 1

    cap.release()
    return count


def process_videos(input_folder, output_folder):
    # Get the list of subdirectories in the input folder
    subdirectories = [d for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]

    # Iterate through each subdirectory
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(input_folder, subdirectory)
        # Iterate over each video file in the subdirectory
        count = 0
        for filename in os.listdir(subdirectory_path):
            if filename.endswith(".MOV"):
                video_path = os.path.join(subdirectory_path, filename)
                video_name = os.path.splitext(filename)[0]
                output_video_folder = os.path.join(output_folder, subdirectory + "/")
                count = extract_frames(count, video_path, output_video_folder)
                print(output_video_folder)
                print(f"Frames extracted from {filename}")
        print(subdirectory_path, count)


In [ ]:
### CHANGE ACCORDINGLY###
# Paths
input_folder = "./0508videos" # The folder of the video
output_folder = "./data/0508_camera_frames_v4" # The folder to save the frames
#### CHANGE ACCORDINGLY###

# Process videos
process_videos(input_folder, output_folder)

./data/0508_camera_frames_v4/0/
Frames extracted from 0.MOV
./0508videos/0 251
./data/0508_camera_frames_v4/1/
Frames extracted from 1.MOV
./0508videos/1 300
./data/0508_camera_frames_v4/2/
Frames extracted from 2.MOV
./0508videos/2 248
./data/0508_camera_frames_v4/3/
Frames extracted from 3.MOV
./0508videos/3 243
./data/0508_camera_frames_v4/4/
Frames extracted from 4.MOV
./0508videos/4 332
./data/0508_camera_frames_v4/5/
Frames extracted from 5.MOV
./0508videos/5 303
./data/0508_camera_frames_v4/6/
Frames extracted from 6.MOV
./0508videos/6 268
./data/0508_camera_frames_v4/7/
Frames extracted from 7.MOV
./0508videos/7 401
./data/0508_camera_frames_v4/8/
Frames extracted from 8.MOV
./0508videos/8 315
./data/0508_camera_frames_v4/9/
Frames extracted from 9.MOV
./0508videos/9 208
./data/0508_camera_frames_v4/10/
Frames extracted from 3&.MOV
./0508videos/10 274
./data/0508_camera_frames_v4/11/
Frames extracted from 9&.MOV
./0508videos/11 269


# Train, Val, Test Split (80-10-10)

In [ ]:
import os
import random
import shutil


def train_val_test_split(root_folder):
  # Loop through each subfolder (class)
  for class_folder in os.listdir(root_folder):
      class_path = os.path.join(root_folder, class_folder)
      print(class_path)

      # Check if the path is a directory
      if os.path.isdir(class_path):
          # Get a list of all images in the class folder
          images = [f for f in os.listdir(class_path)]
          total_images = len(images)
          print(f"Total images in {class_folder}: {total_images}")
          # Randomly shuffle the images
          random.shuffle(images)

          num_train_images = int(total_images * 0.8)
          num_val_images = int(total_images * 0.1)
          num_test_images = total_images - num_train_images - num_val_images

          # Split the images into training and testing sets
          train_images = images[:num_train_images]
          val_images = images[num_train_images : num_train_images+num_val_images]
          test_images = images[num_train_images+num_val_images : num_train_images+num_val_images+num_test_images]

          # Create folders for train and test data if they don't exist
          train_folder = os.path.join(root_folder, 'train', class_folder)
          val_folder = os.path.join(root_folder, 'val', class_folder)
          test_folder = os.path.join(root_folder, 'test', class_folder)
          os.makedirs(train_folder, exist_ok=True)
          os.makedirs(val_folder, exist_ok=True)
          os.makedirs(test_folder, exist_ok=True)

          # Move the images to the respective folders
          for img in train_images:
              src = os.path.join(class_path, img)
              dst = os.path.join(train_folder, img)
              shutil.move(src, dst)

          for img in val_images:
              src = os.path.join(class_path, img)
              dst = os.path.join(val_folder, img)
              shutil.move(src, dst)

          for img in test_images:
              src = os.path.join(class_path, img)
              dst = os.path.join(test_folder, img)
              shutil.move(src, dst)

  print("Data splitting completed successfully.")

./data/0508_camera_frames_v4/0
Total images in 0: 251
./data/0508_camera_frames_v4/1
Total images in 1: 300
./data/0508_camera_frames_v4/2
Total images in 2: 248
./data/0508_camera_frames_v4/3
Total images in 3: 243
./data/0508_camera_frames_v4/4
Total images in 4: 332
./data/0508_camera_frames_v4/5
Total images in 5: 303
./data/0508_camera_frames_v4/6
Total images in 6: 268
./data/0508_camera_frames_v4/7
Total images in 7: 401
./data/0508_camera_frames_v4/8
Total images in 8: 315
./data/0508_camera_frames_v4/9
Total images in 9: 208
./data/0508_camera_frames_v4/10
Total images in 10: 274
./data/0508_camera_frames_v4/11
Total images in 11: 269
Data splitting completed successfully.


In [ ]:
train_val_test_split(output_folder)